# Amazon SageMaker XGBoost를 활용한 다이렉트 마케팅 타겟팅
_**경사 부스팅 트리를 활용한 지도 학습: 불균형 클래스를 가진 이진 예측 문제**_

---

---

## 목차

1. [배경](#Background)
1. [준비](#Preparation)
1. [데이터](#Data)
    1. [탐색](#Exploration)
    1. [변환](#Transformation)
1. [훈련](#Training)
1. [호스팅](#Hosting)
1. [평가](#Evaluation)
1. [확장](#Extensions)

---

## 배경
우편, 이메일, 전화 등을 통한 다이렉트 마케팅은 고객을 확보하는 일반적인 전략입니다. 자원과 고객의 관심이 제한되어 있기 때문에, 목표는 특정 제안에 반응할 가능성이 높은 잠재 고객 하위 집합만을 대상으로 하는 것입니다. 인구통계, 과거 상호작용, 환경적 요소와 같은 쉽게 얻을 수 있는 정보를 기반으로 잠재 고객을 예측하는 것은 일반적인 머신 러닝 문제입니다.

이 노트북은 한 번 이상의 전화 통화 후 고객이 은행의 정기 예금에 가입할지 여부를 예측하는 예제 문제를 제시합니다. 단계는 다음과 같습니다:

* Amazon SageMaker 노트북 준비하기
* 인터넷에서 Amazon SageMaker로 데이터 다운로드하기
* Amazon SageMaker 알고리즘에 공급할 수 있도록 데이터 조사 및 변환하기
* 그래디언트 부스팅 알고리즘을 사용하여 모델 추정하기
* 모델의 효과성 평가하기
* 지속적인 예측을 위한 모델 설정하기

---

## 준비

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성되고 테스트되었습니다._

다음 사항을 지정하는 것으로 시작하겠습니다:

- 훈련 및 모델 데이터에 사용할 S3 버킷과 접두사. 이는 노트북 인스턴스, 훈련 및 호스팅과 동일한 리전에 있어야 합니다.
- 훈련 및 호스팅에 데이터 접근 권한을 부여하는 데 사용되는 IAM 역할 ARN. 생성 방법은 문서를 참조하세요. 참고로, 노트북 인스턴스, 훈련 및/또는 호스팅에 둘 이상의 역할이 필요한 경우 boto regexp를 적절한 전체 IAM 역할 ARN 문자열로 대체하세요.

In [ ]:
# cell 02
import sagemaker
bucket=sagemaker.Session().default_bucket()
prefix = 'sagemaker/DEMO-xgboost-dm'
 
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

이제 분석 전반에 걸쳐 사용할 Python 라이브러리를 가져오겠습니다.

In [ ]:
# cell 03
import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
import matplotlib.pyplot as plt                   # For charts and visualizations
from IPython.display import Image                 # For displaying images in the notebook
from IPython.display import display               # For displaying outputs in the notebook
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs
import os                                         # For manipulating filepath names
import sagemaker 
import zipfile     # Amazon SageMaker's Python SDK provides many helper functions

In [ ]:
# cell 04
pd.__version__

pandas 버전이 1.2.4 이상인지 확인하세요. 그렇지 않은 경우, 계속 진행하기 전에 커널을 재시작하세요.

---

## 데이터
[다이렉트 마케팅 데이터셋](https://sagemaker-sample-data-us-west-2.s3-us-west-2.amazonaws.com/autopilot/direct_marketing/bank-additional.zip)을 샘플 데이터 s3 버킷에서 다운로드하여 시작하겠습니다.

\[Moro et al., 2014\] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014

In [ ]:
# cell 06
data = pd.read_csv('./bank-additional/bank-additional-full.csv')
pd.set_option('display.max_columns', 500)     # Make sure we can see all of the columns
pd.set_option('display.max_rows', 20)         # Keep the output on one page
data

데이터에 대해 이야기해 봅시다. 전체적으로 다음과 같은 사항을 확인할 수 있습니다:

* 약 40,000개의 고객 기록과 각 고객에 대한 20개의 특성이 있습니다
* 특성들은 혼합되어 있으며, 일부는 수치형이고 일부는 범주형입니다
* 데이터는 최소한 `time`과 `contact`에 따라 정렬된 것으로 보이며, 더 많은 기준으로 정렬되었을 수도 있습니다

_**각 특성에 대한 세부 정보:**_

*인구통계학적 정보:*
* `age`: 고객의 나이 (수치형)
* `job`: 직업 유형 (범주형: 'admin.', 'services', ...)
* `marital`: 결혼 상태 (범주형: 'married', 'single', ...)
* `education`: 교육 수준 (범주형: 'basic.4y', 'high.school', ...)

*과거 고객 이벤트:*
* `default`: 신용 불량 여부 (범주형: 'no', 'unknown', ...)
* `housing`: 주택 대출 여부 (범주형: 'no', 'yes', ...)
* `loan`: 개인 대출 여부 (범주형: 'no', 'yes', ...)

*과거 직접 마케팅 접촉:*
* `contact`: 연락 통신 유형 (범주형: 'cellular', 'telephone', ...)
* `month`: 마지막 연락 월 (범주형: 'may', 'nov', ...)
* `day_of_week`: 마지막 연락 요일 (범주형: 'mon', 'fri', ...)
* `duration`: 마지막 연락 지속 시간(초) (수치형). 중요 참고사항: duration = 0이면 `y` = 'no'입니다.
 
*캠페인 정보:*
* `campaign`: 이 캠페인 동안 이 고객에게 수행된 연락 횟수 (수치형, 마지막 연락 포함)
* `pdays`: 이전 캠페인에서 고객에게 마지막으로 연락한 후 경과한 일수 (수치형)
* `previous`: 이 캠페인 이전에 이 고객에게 수행된 연락 횟수 (수치형)
* `poutcome`: 이전 마케팅 캠페인의 결과 (범주형: 'nonexistent', 'success', ...)

*외부 환경 요인:*
* `emp.var.rate`: 고용 변동률 - 분기별 지표 (수치형)
* `cons.price.idx`: 소비자 물가 지수 - 월별 지표 (수치형)
* `cons.conf.idx`: 소비자 신뢰 지수 - 월별 지표 (수치형)
* `euribor3m`: 유리보 3개월 금리 - 일별 지표 (수치형)
* `nr.employed`: 직원 수 - 분기별 지표 (수치형)

*목표 변수:*
* `y`: 고객이 정기 예금에 가입했는가? (이진형: 'yes', 'no')

### 탐색
데이터 준비 위젯에서 데이터 탐색을 시작해보겠습니다. 먼저, 특성들이 어떻게 분포되어 있는지 이해해보겠습니다.

다음 사항에 주목하세요:

* 목표 변수 `y`의 값 중 거의 90%가 "no"이므로, 대부분의 고객은 정기 예금에 가입하지 않았습니다.
* 많은 예측 특성들이 "unknown" 값을 가집니다. 일부는 다른 것보다 더 흔합니다. "unknown" 값이 발생하는 원인(이러한 고객들이 어떤 면에서 대표성이 없는지)과 이를 어떻게 처리해야 하는지 신중하게 생각해야 합니다.
  * "unknown"이 자체적인 별도 카테고리로 포함되더라도, 실제로는 해당 관측치가 해당 특성의 다른 카테고리 중 하나에 속할 가능성이 높은데, 이것이 무엇을 의미하는지 고려해야 합니다.
* 많은 예측 특성들에는 매우 적은 관측치를 가진 카테고리가 있습니다. 작은 카테고리가 목표 결과를 높게 예측한다면, 이에 대한 일반화를 할 만큼 충분한 증거가 있는지 고려해야 합니다.
* 연락 시기는 특히 편향되어 있습니다. 거의 3분의 1이 5월이고 12월은 1% 미만입니다. 이것이 다음 12월에 목표 변수를 예측하는 데 어떤 의미가 있을까요?
* 수치형 특성에는 결측값이 없습니다. 또는 결측값이 이미 대체되었습니다.
  * `pdays`는 거의 모든 고객에 대해 1000에 가까운 값을 가집니다. 이는 이전 접촉이 없음을 나타내는 자리 표시자 값일 가능성이 높습니다.
* 여러 수치형 특성들은 매우 긴 꼬리 분포를 가집니다. 극단적으로 큰 값을 가진 이러한 소수의 관측치를 다르게 처리해야 할까요?
* 여러 수치형 특성(특히 거시경제적 특성)은 뚜렷한 버킷으로 발생합니다. 이들을 범주형으로 취급해야 할까요?

다음으로, 우리가 예측하려는 목표와 특성들이 어떻게 관련되어 있는지 살펴보겠습니다.

In [ ]:
# cell 07
for column in data.select_dtypes(include=['object']).columns:
    if column != 'y':
        print(pd.crosstab(index=data[column], columns=data['y'], normalize='columns'))

for column in data.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = data[[column, 'y']].hist(by='y', bins=30)
    plt.show()

다음 사항에 주목하세요:

* "블루칼라" 직업을 가진 고객, "기혼자", "알 수 없는" 대출 상태인 고객, "전화"로 연락받은 고객, 그리고/또는 "5월"에 연락받은 고객은 구독 "예"보다 "아니오" 비율이 상당히 낮습니다.
* 수치형 변수의 분포는 구독 "예"와 "아니오" 그룹 간에 차이가 있지만, 그 관계는 단순하거나 명확하지 않을 수 있습니다.

이제 특성들이 서로 어떤 관계를 가지는지 살펴보겠습니다.

In [ ]:
# cell 08
print(data.corr(numeric_only=True))
pd.plotting.scatter_matrix(data, figsize=(12, 12))
plt.show()

참고 사항:
* 특성들은 서로의 관계에서 크게 다양합니다. 일부는 매우 높은 음의 상관관계를 가지고, 다른 일부는 매우 높은 양의 상관관계를 가집니다.
* 특성 간의 관계는 많은 경우에 비선형적이고 이산적입니다.

### 변환

데이터 정제는 거의 모든 머신 러닝 프로젝트의 일부입니다. 잘못 수행될 경우 가장 큰 위험을 초래할 수 있으며, 프로세스에서 더 주관적인 측면 중 하나입니다. 몇 가지 일반적인 기법은 다음과 같습니다:

* 결측값 처리: 일부 머신 러닝 알고리즘은 결측값을 처리할 수 있지만, 대부분은 그렇지 않습니다. 옵션은 다음과 같습니다:
 * 결측값이 있는 관측치 제거: 매우 적은 비율의 관측치만 불완전한 정보를 가지고 있을 때 효과적입니다.
 * 결측값이 있는 특성 제거: 많은 결측값을 가진 특성이 소수일 때 효과적입니다.
 * 결측값 대체: 이 주제에 관한 전체 [책](https://www.amazon.com/Flexible-Imputation-Missing-Interdisciplinary-Statistics/dp/1439868247)이 쓰여졌지만, 일반적인 선택은 결측값을 해당 열의 비결측값의 최빈값이나 평균값으로 대체하는 것입니다.
* 범주형을 수치형으로 변환: 가장 일반적인 방법은 원-핫 인코딩으로, 각 특성에 대해 해당 열의 모든 고유 값을 자체 특성으로 매핑하여 범주형 특성이 해당 값과 같을 때 1, 그렇지 않으면 0의 값을 갖도록 합니다.
* 이상하게 분포된 데이터: Gradient Boosted Trees와 같은 비선형 모델의 경우 이것이 매우 제한적인 영향을 미치지만, 회귀와 같은 파라메트릭 모델은 매우 왜곡된 데이터가 주어질 때 매우 부정확한 추정치를 생성할 수 있습니다. 일부 경우에는 특성의 자연 로그를 취하는 것만으로도 더 정규 분포된 데이터를 생성하기에 충분합니다. 다른 경우에는 값을 개별 범위로 버킷팅하는 것이 도움이 됩니다. 이러한 버킷은 범주형 변수로 취급되어 원-핫 인코딩될 때 모델에 포함될 수 있습니다.
* 더 복잡한 데이터 유형 처리: 이미지, 텍스트 또는 다양한 세분화 수준의 데이터 조작은 다른 노트북 템플릿에서 다룹니다.

다행히도 이러한 측면 중 일부는 이미 처리되었으며, 우리가 보여주고자 하는 알고리즘은 희소하거나 이상하게 분포된 데이터를 처리하는 데 잘 작동하는 경향이 있습니다. 따라서 전처리를 간단하게 유지하겠습니다.

In [ ]:
# cell 09

# Note: These transformations can be done through the graphical widget that we generated above. The data prep widget will automatically generate code for transformations that you do.
data['no_previous_contact'] = np.where(data['pdays'] == 999, 1, 0)                                 # Indicator variable to capture when pdays takes a value of 999
data['not_working'] = np.where(np.in1d(data['job'], ['student', 'retired', 'unemployed']), 1, 0)   # Indicator for individuals not actively employed


In [ ]:
# cell 10
model_data = pd.get_dummies(data, dtype=float)                                                                  # Convert categorical variables to sets of indicators

모델을 구축하기 전에 스스로에게 물어봐야 할 또 다른 질문은 특정 특성들이 최종 사용 사례에 가치를 더할 것인지 여부입니다. 예를 들어, 목표가 최상의 예측을 제공하는 것이라면, 예측 시점에 해당 데이터에 접근할 수 있을까요? 비가 오고 있다는 사실은 우산 판매에 매우 예측적이지만, 우산 재고를 계획할 만큼 충분히 먼 날씨를 예측하는 것은 날씨에 대한 지식 없이 우산 판매를 예측하는 것만큼 어려울 수 있습니다. 따라서 이를 모델에 포함시키면 정확도에 대한 잘못된 인식을 가질 수 있습니다.

이러한 논리에 따라, 경제적 특성들과 `duration`을 데이터에서 제거해 보겠습니다. 이러한 특성들은 미래 예측에 입력으로 사용하기 위해서는 높은 정확도로 예측되어야 하기 때문입니다.

경제 지표의 이전 분기 값을 사용하더라도, 이 값은 다음 분기 초반에 연락한 잠재 고객보다 후반에 연락한 잠재 고객에게는 그다지 관련성이 없을 수 있습니다.

In [ ]:
# cell 11
model_data = model_data.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)

새로운 데이터에서 목표값을 예측하는 것이 주요 목적인 모델을 구축할 때, 과적합을 이해하는 것이 중요합니다. 지도 학습 모델은 주어진 데이터에서 목표값에 대한 예측과 실제 값 사이의 오차를 최소화하도록 설계되어 있습니다. 이 마지막 부분이 핵심인데, 더 높은 정확도를 추구하는 과정에서 머신 러닝 모델들은 학습 데이터 내의 사소한 특이점을 포착하는 쪽으로 편향되기 쉽기 때문입니다. 이러한 특이점들은 이후 데이터에서는 반복되지 않기 때문에, 학습 단계에서의 더 정확한 예측을 위해 실제로는 예측의 정확도가 떨어질 수 있습니다.

이를 방지하는 가장 일반적인 방법은 모델이 학습된 데이터에 대한 적합성뿐만 아니라 "새로운" 데이터에 대한 적합성도 평가해야 한다는 개념으로 모델을 구축하는 것입니다. 이를 실현하는 여러 방법이 있습니다. 홀드아웃 검증, 교차 검증, 하나 제외 검증 등이 있습니다. 우리의 목적을 위해, 데이터를 단순히 3개의 불균등한 그룹으로 무작위로 나눌 것입니다. 모델은 데이터의 70%로 학습되고, 그 다음 20%의 데이터로 평가되어 "새로운" 데이터에서 기대할 수 있는 정확도를 추정하며, 10%는 나중에 사용될 최종 테스트 데이터셋으로 남겨둘 것입니다.

In [ ]:
# cell 12
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%

Amazon SageMaker의 XGBoost 컨테이너는 libSVM 또는 CSV 데이터 형식으로 데이터를 받습니다. 이 예제에서는 CSV를 사용하겠습니다. 첫 번째 열은 반드시 타겟 변수여야 하며 CSV에는 헤더가 포함되지 않아야 합니다. 또한 반복적이긴 하지만 train|validation|test 분할 전보다 분할 후에 이 작업을 수행하는 것이 가장 쉽습니다. 이렇게 하면 무작위 재정렬로 인한 정렬 문제를 피할 수 있습니다.

In [ ]:
# cell 13
pd.concat([train_data['y_yes'], train_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['y_yes'], validation_data.drop(['y_no', 'y_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

이제 Amazon SageMaker의 관리형 훈련이 가져갈 수 있도록 파일을 S3에 복사하겠습니다.

In [ ]:
# cell 14
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---

## 실습 1 종료

---

## 훈련
이제 대부분의 특성들이 왜곡된 분포를 가지고 있고, 일부는 서로 높은 상관관계가 있으며, 일부는 목표 변수와 비선형 관계를 가지고 있다는 것을 알게 되었습니다. 또한, 미래의 잠재 고객을 타겟팅하기 위해서는 왜 그 잠재 고객이 타겟팅되었는지 설명하는 것보다 좋은 예측 정확도가 선호됩니다. 이러한 측면들을 종합적으로 고려할 때, 그래디언트 부스팅 트리는 좋은 후보 알고리즘이 됩니다.

알고리즘을 이해하는 데 여러 복잡한 부분이 있지만, 높은 수준에서 그래디언트 부스팅 트리는 이전 모델의 약점을 해결하려고 시도하는 많은 간단한 모델의 예측을 결합하여 작동합니다. 이렇게 함으로써 간단한 모델의 집합이 실제로 크고 복잡한 모델보다 더 나은 성능을 발휘할 수 있습니다. 다른 Amazon SageMaker 노트북에서는 그래디언트 부스팅 트리와 유사한 알고리즘과의 차이점에 대해 더 자세히 설명합니다.

`xgboost`는 그래디언트 부스팅 트리를 위한 매우 인기 있는 오픈 소스 패키지입니다. 이는 계산적으로 강력하고, 기능이 완전하며, 많은 머신 러닝 대회에서 성공적으로 사용되었습니다. Amazon SageMaker의 관리형 분산 훈련 프레임워크를 사용하여 간단한 `xgboost` 모델부터 시작해 보겠습니다.

먼저 Amazon SageMaker의 XGBoost 구현을 위한 ECR 컨테이너 위치를 지정해야 합니다.

In [ ]:
# cell 15
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

그런 다음, CSV 파일 형식으로 훈련하고 있으므로, 훈련 함수가 S3의 파일을 가리키는 포인터로 사용할 수 있는 `s3_input`을 생성하겠습니다. 이는 또한 콘텐츠 타입이 CSV임을 지정합니다.

In [ ]:
# cell 16
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

먼저 estimator에 훈련 매개변수를 지정해야 합니다. 여기에는 다음이 포함됩니다:
1. `xgboost` 알고리즘 컨테이너
1. 사용할 IAM 역할
1. 훈련 인스턴스 유형 및 수량
1. 출력 데이터를 위한 S3 위치
1. 알고리즘 하이퍼파라미터

그리고 다음을 지정하는 `.fit()` 함수:
1. 출력 데이터를 위한 S3 위치. 이 경우 훈련 세트와 검증 세트가 모두 전달됩니다.

In [ ]:
# cell 17
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---

## 호스팅
이제 `xgboost` 알고리즘을 데이터로 훈련시켰으니, 실시간 엔드포인트 뒤에 호스팅되는 모델을 배포해 보겠습니다.

In [ ]:
# cell 18
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

---

## 평가
머신 러닝 모델의 성능을 비교하는 방법은 많지만, 실제 값과 예측 값을 단순히 비교하는 것부터 시작해 보겠습니다. 이 경우, 우리는 고객이 정기 예금에 가입했는지(`1`) 아닌지(`0`)를 예측하는 것이므로 간단한 혼동 행렬(confusion matrix)이 생성됩니다.

먼저 데이터를 엔드포인트로 전달하고 엔드포인트에서 데이터를 받는 방법을 결정해야 합니다. 현재 데이터는 노트북 인스턴스의 메모리에 NumPy 배열로 저장되어 있습니다. HTTP POST 요청으로 전송하기 위해 CSV 문자열로 직렬화한 다음 결과 CSV를 디코딩하겠습니다.

*참고: CSV 형식으로 추론할 때 SageMaker XGBoost는 데이터에 타겟 변수가 포함되지 않아야 합니다.*

In [ ]:
# cell 19
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

이제 다음과 같은 간단한 함수를 사용하여:
1. 테스트 데이터셋을 반복하고
1. 행을 미니 배치로 분할하고
1. 이러한 미니 배치를 CSV 문자열 페이로드로 변환하고(참고로, 먼저 데이터셋에서 타겟 변수를 제거합니다)
1. XGBoost 엔드포인트를 호출하여 미니 배치 예측을 검색하고
1. 예측을 수집하고 모델이 제공하는 CSV 출력을 NumPy 배열로 변환합니다.

In [ ]:
# cell 20
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(), xgb_predictor)

이제 혼동 행렬을 확인하여 예측과 실제 값을 얼마나 잘 예측했는지 살펴보겠습니다.

In [ ]:
# cell 21
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

따라서 약 4000명의 잠재 고객 중에서 136명이 가입할 것으로 예측했고 그 중 94명이 실제로 가입했습니다. 또한 우리가 예측하지 못한 389명의 가입자가 실제로 가입했습니다. 이는 바람직하지 않지만, 모델을 튜닝하여 개선할 수 있습니다. 가장 중요한 점은 최소한의 노력으로 우리 모델이 [여기](https://core.ac.uk/download/pdf/55631291.pdf)에 게시된 것과 유사한 정확도를 생성했다는 것입니다.

_알고리즘의 하위 샘플에 무작위성 요소가 있기 때문에 결과가 위에 작성된 텍스트와 약간 다를 수 있습니다._

### 정리
이 노트북에서 생성한 리소스 중 더 이상 사용하지 않을 리소스를 삭제하세요.

In [ ]:
# cell 22
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

--
## 서버리스 배포 (선택 사항)
모델 훈련 후, 모델 아티팩트를 검색하여 엔드포인트에 모델을 배포할 수 있습니다.

In [ ]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [ ]:
# Retrieve model data from training job
model_artifacts = xgb.model_data
model_artifacts

### 모델 생성
모델 아티팩트, 컨테이너 이미지 URI, 컨테이너의 환경 변수(해당되는 경우), 모델 이름 및 SageMaker IAM 역할을 제공하여 모델을 생성합니다.

In [ ]:
from time import gmtime, strftime

model_name = "xgboost-serverless" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name: " + model_name)

# dummy environment variables
byo_container_env_vars = {"SAGEMAKER_CONTAINER_LOG_LEVEL": "20", "SOME_ENV_VAR": "myEnvVar"}

create_model_response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            "Image": container,
            "Mode": "SingleModel",
            "ModelDataUrl": model_artifacts,
            "Environment": byo_container_env_vars,
        }
    ],
    ExecutionRoleArn=role,
)

print("Model Arn: " + create_model_response["ModelArn"])

### 엔드포인트 구성 생성
여기서 엔드포인트에 대한 서버리스 구성을 조정할 수 있습니다. 단일 엔드포인트에 대한 현재 최대 동시 호출 수인 MaxConcurrency는 1에서 200 사이의 값이 될 수 있으며, MemorySize는 다음 중 하나가 될 수 있습니다: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB 또는 6144 MB.

In [ ]:
xgboost_epc_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=xgboost_epc_name,
    ProductionVariants=[
        {
            "VariantName": "byoVariant",
            "ModelName": model_name,
            "ServerlessConfig": {
                "MemorySizeInMB": 4096,
                "MaxConcurrency": 1,
            },
        },
    ],
)

print("Endpoint Configuration Arn: " + endpoint_config_response["EndpointConfigArn"])

### 서버리스 엔드포인트 생성
이제 엔드포인트 구성이 있으므로 서버리스 엔드포인트를 생성하고 모델을 배포할 수 있습니다. 엔드포인트를 생성할 때 엔드포인트 구성 이름과 새 엔드포인트의 이름을 제공합니다.

In [ ]:
endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=xgboost_epc_name,
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

엔드포인트를 호출하기 전에 엔드포인트 상태가 InService가 될 때까지 기다립니다.

In [ ]:
# wait for endpoint to reach a terminal state (InService) using describe endpoint
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

### 엔드포인트 호출
엔드포인트에 요청을 보내 호출합니다. 다음은 Direct Marketing 데이터셋에서 다운로드한 CSV 파일에서 가져온 샘플 데이터 포인트입니다.

In [ ]:
payload ="29,2,999,0,1,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0"
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=payload,
    ContentType="text/csv",
)

print(response["Body"].read())

### 정리
이 노트북에서 생성한 리소스 중 더 이상 사용하지 않을 리소스를 삭제하세요.

In [ ]:
client.delete_model(ModelName=model_name)
client.delete_endpoint_config(EndpointConfigName=xgboost_epc_name)
client.delete_endpoint(EndpointName=endpoint_name)

## 자동 모델 튜닝 (선택 사항)
Amazon SageMaker 자동 모델 튜닝(하이퍼파라미터 튜닝이라고도 함)은 지정한 알고리즘 및 하이퍼파라미터 범위를 사용하여 데이터셋에서 많은 훈련 작업을 실행하여 모델의 최상의 버전을 찾습니다. 그런 다음 선택한 메트릭으로 측정했을 때 가장 좋은 성능을 보이는 모델을 생성하는 하이퍼파라미터 값을 선택합니다.
예를 들어, 이 마케팅 데이터셋에서 이진 분류 문제를 해결하려고 한다고 가정해 보겠습니다. 목표는 XGBoost 알고리즘 모델을 훈련하여 알고리즘의 곡선 아래 영역(auc) 메트릭을 최대화하는 것입니다. 최상의 모델을 훈련하기 위해 eta, alpha, min_child_weight 및 max_depth 하이퍼파라미터의 어떤 값을 사용해야 하는지 모릅니다. 이러한 하이퍼파라미터에 대한 최상의 값을 찾기 위해 Amazon SageMaker 하이퍼파라미터 튜닝이 선택한 목표 메트릭으로 측정했을 때 가장 좋은 성능을 보이는 훈련 작업을 생성하는 값 조합을 찾기 위해 검색할 값 범위를 지정할 수 있습니다. 하이퍼파라미터 튜닝은 지정한 범위의 하이퍼파라미터 값을 사용하는 훈련 작업을 시작하고 auc가 가장 높은 훈련 작업을 반환합니다.

In [ ]:
# cell 22
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}


In [ ]:
# cell 23
objective_metric_name = 'validation:auc'

In [ ]:
# cell 24
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=20,
                            max_parallel_jobs=3)


In [ ]:
# cell 25
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

In [ ]:
# cell 26
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

In [ ]:
# cell 27
# return the best training job name
tuner.best_training_job()

In [ ]:
# cell 28
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

In [ ]:
# cell 29
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# cell 30
# Predict
predictions = predict(test_data.drop(['y_no', 'y_yes'], axis=1).to_numpy(),tuner_predictor)

In [ ]:
# cell 31
# Collect predictions and convert from the CSV output our model provides into a NumPy array
pd.crosstab(index=test_data['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

---

## 확장

이 예제는 비교적 작은 데이터셋을 분석했지만 분산 관리형 훈련 및 실시간 모델 호스팅과 같은 Amazon SageMaker 기능을 활용했으며, 이는 훨씬 더 큰 문제에도 쉽게 적용할 수 있습니다. 예측 정확도를 더욱 향상시키기 위해 예측 임계값을 조정하여 거짓 양성과 거짓 음성의 비율을 변경하거나 하이퍼파라미터 튜닝과 같은 기술을 탐색할 수 있습니다. 실제 시나리오에서는 수작업으로 특성 엔지니어링에 더 많은 시간을 할애하고 초기 데이터셋에서 사용할 수 없는 고객 정보가 포함된 추가 데이터셋을 찾을 가능성이 높습니다.

### 정리

이 노트북을 마쳤다면 아래 셀을 실행해 주세요. 이렇게 하면 생성한 호스팅 엔드포인트가 제거되고 인스턴스가 실행된 채로 남아 요금이 발생하는 것을 방지할 수 있습니다.

In [ ]:
# cell 33
tuner_predictor.delete_endpoint(delete_endpoint_config=True)